# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [3]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [4]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames)
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames)
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames)

    return users, ratings, movies

In [5]:
users, ratings, movies = get_movie_data()

C:\Users\harin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\Users\harin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
C:\Users\harin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


In [6]:
users.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [9]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

0    Toy Story
1         1995
dtype: object

In [10]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [11]:
movies.head()

,movie_id,title,genres,year,short_title
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


##### Join the tables with `pd.merge` (20 pts)

## Merge Users and ratings first since it is a one-to-many relationship 

In [12]:
#merges users and ratings based on user_id
UserRatingMerge = pd.merge(users,ratings, on = ["user_id"])
UserRatingMerge.head()

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp
0,1,F,1,10,48067,1193,5,978300760
1,1,F,1,10,48067,661,3,978302109
2,1,F,1,10,48067,914,3,978301968
3,1,F,1,10,48067,3408,4,978300275
4,1,F,1,10,48067,2355,5,978824291


## Merge the merged dataset with movies dataset to get a super merged dataframe. 

In [13]:
SuperMovie = pd.merge(movies,UserRatingMerge, on = ["movie_id"])
#sorts by user id to make sure merge was correct and that all users are in the dataset. 
SuperMovie.sort_values(by=['user_id']).head()

,movie_id,title,genres,year,short_title,user_id,gender,age,occupation,zip,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Toy Story,1,F,1,10,48067,5,978824268
310576,1207,To Kill a Mockingbird (1962),Drama,1962,To Kill a Mockingbird,1,F,1,10,48067,4,978300719
341051,1246,Dead Poets Society (1989),Drama,1989,Dead Poets Society,1,F,1,10,48067,4,978302091
745569,2762,"Sixth Sense, The (1999)",Thriller,1999,"Sixth Sense, The",1,F,1,10,48067,4,978302091
144754,531,"Secret Garden, The (1993)",Children's|Drama,1993,"Secret Garden, The",1,F,1,10,48067,4,978302149


##### What's the highest rated movie? (20 pts))

### Gets an average rating of all movies and determines the size which indicates how many times people have rated the movie. (higher size = better indication of rating)

In [45]:
RatedMov = SuperMovie[['title','rating']]
#Gets average of ratings per movie
AvgRatedMov = RatedMov.groupby(['title']).mean()
#Creates a new column (Size) which finds the amount of people who rated the movie
AvgRatedMov['Size'] = RatedMov.groupby(['title']).size().to_frame('size')

#finds most rated movie to get a better range. 
AvgRatedMov.sort_values(by = ['Size'], ascending = False).head()
# The highest amount of times a movie has been is 3428 times
# We will look for movies over 1000 size then when finding the highly rated movie. 

,rating,Size
title,,
American Beauty (1999),4.317386,3428
Star Wars: Episode IV - A New Hope (1977),4.453694,2991
Star Wars: Episode V - The Empire Strikes Back (1980),4.292977,2990
Star Wars: Episode VI - Return of the Jedi (1983),4.022893,2883
Jurassic Park (1993),3.763847,2672


### Display movies who have been rated at least a 1000 times and finds highly rated Movie. 

In [15]:
#only count the movies that have been rated at least 1000 times by users
Top1000Movies = AvgRatedMov[AvgRatedMov['Size']>=1000]
#since we have movies that are rated over 1000 times, sort by ratings. 
HighRate_Mov = Top1000Movies.sort_values(['rating'],ascending = False)

#display and the top rating should be the highly rated movie. 
HighRate_Mov.head()

,rating,Size
title,,
"Shawshank Redemption, The (1994)",4.554558,2227
"Godfather, The (1972)",4.524966,2223
"Usual Suspects, The (1995)",4.517106,1783
Schindler's List (1993),4.510417,2304
Raiders of the Lost Ark (1981),4.477725,2514


## ANSWER: The Shawshank Redemption is the highly rated movie. 

###### What is a good rated movie for date night? (60 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined

## Remove unwanted variables and calculate the amount of times the movie has been reviewed. 

In [155]:
#remove unwanted variables that have no contributation in finding a movie for date night. 
Short_mymov = SuperMovie.drop(["movie_id","user_id","short_title","occupation","zip","timestamp","year"], axis = 1)

#Calculate the size of how many times the movie has been reviewd. 
Rates['Size'] = Short_mymov.groupby(['title']).size().to_frame('size')

## Select desired Genre. 

In [198]:
DesiredMov = Rates[Rates['genres'].str.contains('Comedy')]

## Group the variables together by title and genre, giving an average rating and age group of both genders. 

In [199]:
#group tableset by title, genre.
#this will give an average rating of and age group of both genders together.
Avg = DesiredMov.groupby(['title','genres']).mean()

#sortby size to estimate the cutoff point where we decide the highly rated movie. 
Avg.sort_values(by = ['Size'], ascending = False).head()

,,age,rating,Size
title,genres,,,
American Beauty (1999),Comedy|Drama,29.615228,4.317386,3428.0
Back to the Future (1985),Comedy|Sci-Fi,29.895470,3.990321,2583.0
Men in Black (1997),Action|Adventure|Comedy|Sci-Fi,28.941686,3.739953,2538.0
Shakespeare in Love (1998),Comedy|Romance,30.564373,4.127480,2369.0
"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,28.084556,4.303710,2318.0


## Only allow movies that have a size of 1000 or greater and set average age range to mid 20's. 

In [200]:
#only count the movies that have been rated at least 1000 times by users
Top1000 = Avg[Avg['Size']>=1000]

#Set age group to mid 20's. 
MyDesiredMov = Top1000[(Top1000['age'] >= 20) & (Top1000['age'] < 28)]
MyDesiredMov

,,age,rating,Size
title,genres,,,
Aladdin (1992),Animation|Children's|Comedy|Musical,26.297557,3.788305,1351.0
American Pie (1999),Comedy,25.164147,3.709863,1389.0
Austin Powers: International Man of Mystery (1997),Comedy,25.788382,3.710373,1205.0
Austin Powers: The Spy Who Shagged Me (1999),Comedy,26.319386,3.388424,1434.0
Batman Returns (1992),Action|Adventure|Comedy|Crime,26.641125,2.976722,1031.0
"Bug's Life, A (1998)",Animation|Children's|Comedy,27.964768,3.854375,1703.0
Clerks (1994),Comedy,25.321530,3.946884,1412.0
Clueless (1995),Comedy|Romance,27.286344,3.623348,1362.0
Election (1999),Comedy,27.539422,3.930355,1522.0


## Sort Movies by highest rated. 

In [201]:
#sort movies by highest rating
DateNight_Mov = MyDesiredMov.sort_values(['rating'],ascending = False)
DateNight_Mov[:5]

,,age,rating,Size
title,genres,,,
Toy Story 2 (1999),Animation|Children's|Comedy,27.743218,4.218927,1585.0
Toy Story (1995),Animation|Children's|Comedy,27.700530,4.146846,2077.0
Ferris Bueller's Day Off (1986),Comedy,27.949762,4.117447,1473.0
Clerks (1994),Comedy,25.321530,3.946884,1412.0
Election (1999),Comedy,27.539422,3.930355,1522.0


# ANSWER: Toy Story 2 would be a good rated movie for date night. 